In [1]:
import pandas as pd
from pytrends.request import TrendReq
import datetime
import time

In [2]:
keyWords = ["Apartment","Home", "Lending","Real Estate Bubble","Broker","Home Equity","Mortgage", "Real Estate Investment","Bubble", "Housing Bubble", "Real Estate","Real Estate Listings","Construction","Housing Market","Real Estate Agent","Realtor","Flat","Investment","Real Estate Broker","Rent", "zillow", "redfin"]

In [22]:
def GVData(kw, months):
    pytrend = TrendReq(backoff_factor=0.1)
    kw_list = [kw]
    # get your initial data
    full_year = "{} {}".format(months[0],months[1])
    pytrend.build_payload(kw_list, cat=29, timeframe=full_year, geo='US') 
    #cat 29 corresponds to Realestate category see: https://github.com/pat310/google-trends-api/wiki/Google-Trends-Categories , timeframe corresponds to 2004-present
    df_state = pytrend.interest_by_region() # Returns a pandas.dataframe of geoName and gt index [0, 100]
    # add YEAR and STATES column to data frame
    df_state["year"] = [months[0]]*51
    lastN = 0
    kStart = 1
    while(lastN < len(months)-2):
        try:
            for n,m in enumerate(months[kStart:-1]):        
                full_year = "{} {}".format(m,months[n+kStart+1])
                pytrend.build_payload(kw_list, cat=29, timeframe=full_year, geo='US') #cat 29 corresponds to Realestate category see: https://github.com/pat310/google-trends-api/wiki/Google-Trends-Categories , timeframe corresponds to 2004-present
                df = pytrend.interest_by_region()
                # add the year and states column
                year = [m] * 51
                df["year"] = year
                #df["states"] = states 
                # append df to df_state
                df_state = df_state.append(df)
                lastN = n+ kStart
                print(m)
        except:
            print("Error", lastN)
        
        if(lastN < len(months)-2):
            kStart = lastN+1
            print("Sleeping for 60s")
            time.sleep(60)
            
    return df_state

In [11]:
dt = pd.date_range(start='01/01/2005', end='31/12/2020', freq='M')
months = [d.strftime('%Y-%m-%d') for d in dt]

for kw in keyWords:
    df = GVData(kw, months)
    fileName = "./GSVI/GSVI_" + '_'.join(kw.split()) + ".csv"
    df.to_csv(fileName)
    print(kw)

2007-02-28
2007-03-31
2007-04-30
2007-05-31
2007-06-30
2007-07-31
2007-08-31
2007-09-30
2007-10-31
2007-11-30
2007-12-31
2008-01-31
2008-02-29
2008-03-31
2008-04-30
2008-05-31
2008-06-30
2008-07-31
2008-08-31
2008-09-30
2008-10-31
2008-11-30
2008-12-31
2009-01-31
2009-02-28
2009-03-31
2009-04-30
2009-05-31
2009-06-30
2009-07-31
2009-08-31
2009-09-30
2009-10-31
2009-11-30
2009-12-31
2010-01-31
2010-02-28
2010-03-31
2010-04-30
2010-05-31
2010-06-30
2010-07-31
2010-08-31
2010-09-30
2010-10-31
2010-11-30
2010-12-31
2011-01-31
2011-02-28
2011-03-31
2011-04-30
2011-05-31
2011-06-30
2011-07-31
2011-08-31
2011-09-30
2011-10-31
2011-11-30
2011-12-31
2012-01-31
2012-02-29
2012-03-31
2012-04-30
2012-05-31
2012-06-30
2012-07-31
2012-08-31
2012-09-30
2012-10-31
2012-11-30
2012-12-31
2013-01-31
2013-02-28
2013-03-31
2013-04-30
2013-05-31
2013-06-30
2013-07-31
2013-08-31
2013-09-30
2013-10-31
2013-11-30
2013-12-31
2014-01-31
2014-02-28
2014-03-31
2014-04-30
2014-05-31
2014-06-30
2014-07-31
2014-08-31

In [57]:
import glob
gsviFiles = glob.glob("./GSVI/GSVI_*.csv")

In [58]:
if(len(gsviFiles) > 0):
    df = pd.read_csv(gsviFiles[0])
    for file in gsviFiles[1:]:
        print(file)
        dfN = pd.read_csv(file)
        df = df.merge(dfN, how='outer', on = ['year','geoName'])

./GSVI\GSVI_Broker.csv
./GSVI\GSVI_Bubble.csv
./GSVI\GSVI_Construction.csv
./GSVI\GSVI_Flat.csv
./GSVI\GSVI_Home.csv
./GSVI\GSVI_Home_Equity.csv
./GSVI\GSVI_Housing_Bubble.csv
./GSVI\GSVI_Housing_Market.csv
./GSVI\GSVI_Investment.csv
./GSVI\GSVI_Lending.csv
./GSVI\GSVI_Mortgage.csv
./GSVI\GSVI_Realtor.csv
./GSVI\GSVI_Real_Estate.csv
./GSVI\GSVI_Real_Estate_Agent.csv
./GSVI\GSVI_Real_Estate_Broker.csv
./GSVI\GSVI_Real_Estate_Bubble.csv
./GSVI\GSVI_Real_Estate_Investment.csv
./GSVI\GSVI_Real_Estate_Listings.csv
./GSVI\GSVI_redfin.csv
./GSVI\GSVI_Rent.csv
./GSVI\GSVI_zillow.csv


In [59]:
df.to_csv("./GSVI/merged_GSVI.csv", index=False)

In [ ]:
#seasonality correction
#Zillow data is already correct for A smoothed, seasonally adjusted measure of the typical home value and market changes across a given region and housing typ 
#https://www.zillow.com/research/data-research/zillow-home-value-index/

#correction for inflation



In [3]:
# google index
gsvi = pd.read_csv('./GSVI/merged_GSVI.csv')
gsvi.head()

,geoName,Apartment,year,Broker,Bubble,Construction,Flat,Home,Home Equity,Housing Bubble,...,Realtor,Real Estate,Real Estate Agent,Real Estate Broker,Real Estate Bubble,Real Estate Investment,Real Estate Listings,redfin,Rent,zillow
0,Alabama,92,2007-01-31,0,0,18,0,76,0,0,...,43,47,42,0,0,66,71,0,49,0
1,Alaska,0,2007-01-31,0,0,0,100,44,0,0,...,55,30,0,0,0,0,0,0,9,38
2,Arizona,61,2007-01-31,19,0,7,26,68,0,0,...,58,49,18,16,0,56,24,0,95,100
3,Arkansas,13,2007-01-31,0,0,0,0,49,0,0,...,18,55,0,0,0,0,0,0,53,25
4,California,66,2007-01-31,20,26,11,14,50,0,11,...,43,38,14,10,14,25,15,4,61,68


In [20]:
z = gsvi[keyWords].apply(lambda x: x==0).sum() > gsvi.shape[0]*0.25

In [21]:
z.index[z==True]

Index(['Lending', 'Real Estate Bubble', 'Broker', 'Home Equity',
       'Real Estate Investment', 'Bubble', 'Housing Bubble', 'Housing Market',
       'Flat', 'Real Estate Broker', 'redfin'],
      dtype='object')

In [ ]:
topState = ['California', 'Nevada', 'Arizona', 'Florida']
bottomState = ['New York', 'Texas', 'Illinois', 'Pennsylvania']